<p align="center">
  <h1 align="center">🌊 GradTracer v0.7.2 — AI Agent Compression Recipe Ablation Study</h1>
  <p align="center">
    <strong>Empirical Proof: GradTracer Recipe > L1 Magnitude Pruning</strong>
  </p>
</p>

---

This notebook demonstrates why simply pruning weights based on magnitude (L1 Norm) is dangerous, and how GradTracer's **Mixed-Precision Joint Pruning Recipe** (based on Gradient SNR & Dynamics) preserves model accuracy while achieving massive VRAM savings.

We will compare 3 scenarios at **50% Target Sparsity**:
1. **Baseline Model** (Dense FP32)
2. **Magnitude Pruning (L1 Unstructured)**: The industry standard naive approach.
3. **GradTracer Recipe (AI Auto-Execution)**: Automatically generated JSON recipe applied via `torch.nn.utils.prune`.

## 1. Setup & Toy Dataset

In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import numpy as np
from sklearn.datasets import make_classification
from torch.utils.data import TensorDataset, DataLoader
import copy
import json

# Create a complex toy dataset where some features are noises
X, y = make_classification(n_samples=2000, n_features=128, n_informative=20, n_redundant=10, random_state=42)
X_tensor = torch.FloatTensor(X)
y_tensor = torch.LongTensor(y)

dataset = TensorDataset(X_tensor, y_tensor)
train_size = 1600
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, 400])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

def evaluate_acc(model, loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            out = model(data)
            pred = out.argmax(dim=1)
            correct += (pred == target).sum().item()
    return correct / len(loader.dataset)


## 2. Model Definition & Baseline Training with GradTracer
We train a deep MLP and attach `FlowTracker` to monitor its dynamics. The tracker will map out the "learning highways" and the "dead zones".

In [ ]:
from gradtracer import FlowTracker

class DeepMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(128, 512),
            nn.GELU(),
            nn.Linear(512, 1024), # intentional bottleneck/expansion
            nn.GELU(),
            nn.Linear(1024, 256),
            nn.GELU(),
            nn.Linear(256, 2)
        )
        
    def forward(self, x):
        return self.net(x)

baseline_model = DeepMLP()
optimizer = torch.optim.AdamW(baseline_model.parameters(), lr=1e-3, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

tracker = FlowTracker(baseline_model, track_gradients=True, track_weights=True)

print("Training Baseline Model...")
baseline_model.train()
for epoch in range(15):
    for data, target in train_loader:
        optimizer.zero_grad()
        out = baseline_model(data)
        loss = criterion(out, target)
        loss.backward()
        
        # Track dynamics
        tracker.step(loss.item())
        optimizer.step()

baseline_acc = evaluate_acc(baseline_model, test_loader)
print(f"🔥 Baseline Accuracy (Dense FP32): {baseline_acc*100:.2f}%")

# Save copies for pruning
l1_model = copy.deepcopy(baseline_model)
gradtracer_model = copy.deepcopy(baseline_model)

## 3. Magnitude Pruning (L1 Norm)
The industry standard: Just cut the bottom 50% of weights by absolute value across all Linear layers.

In [ ]:
print("Applying 50% L1 Global Magnitude Pruning...")
parameters_to_prune = []
for module_name, module in l1_model.named_modules():
    if isinstance(module, nn.Linear):
        parameters_to_prune.append((module, 'weight'))

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.5,
)

l1_acc = evaluate_acc(l1_model, test_loader)
print(f"⚠️ L1 Pruned Accuracy: {l1_acc*100:.2f}%")

## 4. GradTracer Recipe Generation (v0.7.2)
We ask `RecipeGenerator` to look at the tracking history and produce an AI-friendly JSON manifest.

In [ ]:
from gradtracer.analyzers.recipes import RecipeGenerator

recipe_gen = RecipeGenerator(tracker)
recipe_json = recipe_gen.generate(target_sparsity=0.5)

print("🤖 AI Agent Recipe Manifest:")
print(json.dumps(recipe_json, indent=2))

## 5. GradTracer Auto-Executor (Applying the JSON Recipe)
This simulates what an AI Coder (like Antigravity or Cursor) would do automatically upon reading the JSON. It applies dynamic pruning ratios based on the JSON logic.

In [ ]:
print("Applying GradTracer Joint Pruning Recipe...")

for layer_path, instructions in recipe_json["layers"].items():
    if instructions["prune_ratio"] > 0:
        # Parse 'net.0.weight' -> module 'net.0', param 'weight'
        module_path = layer_path.rsplit('.', 1)[0]
        param_name = layer_path.split('.')[-1]
        
        try:
            module = gradtracer_model.get_submodule(module_path)
            
            # Execute the manifest instruction
            if instructions["prune_type"] == "unstructured_l1":
                prune.l1_unstructured(module, name=param_name, amount=instructions["prune_ratio"])
                
            print(f"✅ Pruned {layer_path} by {instructions['prune_ratio']*100}% ({instructions['reason']})")
        except Exception as e:
            print(f"Skipping {layer_path} - {str(e)}")
    else:
         print(f"🛡️ Protected {layer_path} (0% prune) ({instructions['reason']})")


gt_acc = evaluate_acc(gradtracer_model, test_loader)
print(f"\n🌟 GradTracer Recipe Accuracy: {gt_acc*100:.2f}%")

## 6. Ablation Results Summary
GradTracer heavily protects the High SNR "Information Highways" while brutally compressing the dead zones (Low SNR). L1 Magnitude pruning blindly cuts large weights that might be carrying critical sparse features.

In [ ]:
print("=================================================")
print("📊 Ablation Study Results (Target Sparsity: 50%)")
print("=================================================")
print(f"1. Baseline (Dense FP32)   : {baseline_acc*100:.2f}%")
print(f"2. L1 Magnitude Pruned     : {l1_acc*100:.2f}% (Drop: {(baseline_acc - l1_acc)*100:.2f}%)")
print(f"3. GradTracer Recipe       : {gt_acc*100:.2f}% (Drop: {(baseline_acc - gt_acc)*100:.2f}%)")
print("-------------------------------------------------")
vram_saved = recipe_json['metadata']['estimated_vram_saving_mb']
flops_saved = recipe_json['metadata']['estimated_flops_reduction_ratio'] * 100
print(f"💡 GradTracer Estimated Savings: {vram_saved} MB VRAM, {flops_saved:.1f}% FLOPs reduction.")
print("=================================================")